In [1]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext snakeviz
%load_ext cython

In [135]:
%%cython
def build_faces(long [:,:] tetras, long [:,:] faces, long [:,:] face_tetra):
    cdef int i, j, k, l, n
    cdef int fc = 0
    cdef int ntetra = tetras.shape[0]
    for i in range(ntetra):
        for j in range(i+1,ntetra):
            n = 0
            for k in range(4):
                for l in range(4):
                    if tetras[i,k] == tetras[j,l]:
                        faces[fc,n] = tetras[i,k]
                        face_tetra[fc,0] = i
                        face_tetra[fc,1] = j
                        n+=1

            if n==3:
                fc+=1
                continue
    return fc

def build_neighbours(long [:,:] tetras, long [:,:] neighbours):
    cdef int i, j, k, l, n, nn
    cdef int ntetra = tetras.shape[0]
    for i in range(ntetra):
        nn = 0
        for j in range(ntetra):
            if j == i:
                continue
            n = 0
            for k in range(4):
                for l in range(4):
                    if tetras[i,k] == tetras[j,l]:
                        n+=1
            if n==3:
                neighbours[i,nn] = j
#                 print('n==3',i,nn,j)
                nn+=1
            if nn == 4:
#                 print('nn==3',i,j,n,nn)
                break
            if n == 2:
                continue

            
    return

In [185]:
class TetMesh:
    def __init__(self,origin,nsteps,step_vector):
        self.origin = np.array(origin) 
        self.step_vector = np.array(step_vector)     
        self.nsteps = np.array(nsteps)
        self.nsteps_cells = self.nsteps - 1
        self.n_cell_x = self.nsteps[0] - 1
        self.n_cell_y = self.nsteps[1] - 1
        self.n_cell_z = self.nsteps[2] - 1
        self.n_cells = self.n_cell_x*self.n_cell_y*self.n_cell_z
#         self.tetra_mask = np.array([
#                       [0,6,5,4],
#                       [0,4,5,3],
#                       [4,6,7,5],
#                       [0,1,6,4],
#                       [0,6,5,2]])
        sel
        self.ntetra = self.n_cells*5
    def get_tetra(self,pos):
        pos = np.array(pos)
        # find which cell the points are in
        gi, inside = self.position_to_cell_corners(pos)
        
        # convert global corners to local index
        xi,yi,zi = self.global_index_to_node_index(gi)
        
        # convert to node locations
        nodes = self.node_indexes_to_position(xi,yi,zi).T

        points=nodes[self.tetra_mask,:,:]
        # changing order to points, tetra, nodes, coord
        points = points.swapaxes(0,2)
        points = points.swapaxes(1,2)
        
        # use scalar triple product to calculate barycentric coords
        vap = pos[:,None,:] - points[:,:, 0,:]
        vbp = pos[:,None,:] - points[:,:, 1, :]
        #         # vcp = p - points[:, 2, :]
        #         # vdp = p - points[:, 3, :]
        vab = points[:,:,  1, :] - points[ :,:, 0, :]
        vac = points[:,:,  2, :] - points[ :,:, 0, :]
        vad = points[:,:,  3, :] - points[ :,:, 0, :]
        vbc = points[:,:,  2, :] - points[:, :, 1, :]
        vbd = points[:,:,  3, :] - points[:, :, 1, :]
        va = np.einsum('ikj, ikj->ik', vbp, np.cross(vbd, vbc, axisa=2, axisb=2)) / 6.
        vb = np.einsum('ikj, ikj->ik', vap, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
        vc = np.einsum('ikj, ikj->ik', vap, np.cross(vad, vab, axisa=2, axisb=2)) / 6.
        vd = np.einsum('ikj, ikj->ik', vap, np.cross(vab, vac, axisa=2, axisb=2)) / 6.
        v = np.einsum('ikj, ikj->ik', vab, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
        c = np.zeros((va.shape[0],va.shape[1],4))
            #print(va.shape)
        c[:,:,0] = va / v
        c[:,:,1]= vb / v
        c[:,:,2] = vc / v
        c[:,:,3] = vd / v
        
        # if all coords are +ve then point is inside cell
        mask = np.all(c > 0,axis=2)
        return points[mask,:,:], c[mask]

    def calc_bary_centre(self,points,position):

        npts = len(e)
        vap = position - points[:, 0, :]
        vbp = position - points[:, 1, :]
        # vcp = p - points[:, 2, :]
        # vdp = p - points[:, 3, :]
        vab = points[:, 1, :] - points[:, 0, :]
        vac = points[:, 2, :] - points[:, 0, :]
        vad = points[:, 3, :] - points[:, 0, :]
        vbc = points[:, 2, :] - points[:, 1, :]
        vbd = points[:, 3, :] - points[:, 1, :]
        vbp * np.cross(vbd, vbc, axisa=1, axisb=1)
        va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
        vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
        vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
        v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        c = np.zeros((4, npts))
        c[0, :] = va / v
        c[1, :] = vb / v
        c[2, :] = vc / v
        c[3, :] = vd / v
        return c
    
    
    def inside(self, pos):

        # check whether point is inside box
        inside = np.ones(pos.shape[0]).astype(bool)
        for i in range(3):
            inside *= pos[:, i] > self.origin[None, i]
            inside *= pos[:, i] < self.origin[None, i] + \
                      self.step_vector[None, i] * self.nsteps_cells[None, i]
        return inside    
    def global_node_indicies(self,indexes):
        indexes = np.array(indexes).swapaxes(0, 2)
        return indexes[:, :, 0] + self.nsteps[None, None, 0] \
                * indexes[:, :,1] + self.nsteps[None, None, 0] * \
                self.nsteps[None, None, 1] * indexes[:, :, 2]
    
    def global_cell_indicies(self,indexes):
        indexes = np.array(indexes).swapaxes(0, 2)
        return indexes[:, :, 0] + self.nsteps_cells[None, None, 0] \
                * indexes[:, :,1] + self.nsteps_cells[None, None, 0] * \
                self.nsteps_cells[None, None, 1] * indexes[:, :, 2]
    def cell_corner_indexes(self, x_cell_index, y_cell_index, z_cell_index):
        """
        Returns the indexes of the corners of a cell given its location xi,
        yi, zi

        Parameters
        ----------
        x_cell_index
        y_cell_index
        z_cell_index

        Returns
        -------

        """
        xcorner = np.array([0, 1, 0, 0, 1, 0, 1, 1])
        ycorner = np.array([0, 0, 1, 0, 0, 1, 1, 1])
        zcorner = np.array([0, 0, 0, 1, 1, 1, 0, 1])
        xcorners = x_cell_index[:, None] + xcorner[None, :]
        ycorners = y_cell_index[:, None] + ycorner[None, :]
        zcorners = z_cell_index[:, None] + zcorner[None, :]
        return xcorners, ycorners, zcorners
    
    def position_to_cell_corners(self,pos):
        inside = self.inside(pos)
        ix, iy, iz = self.position_to_cell_index(pos)
        cornersx, cornersy, cornersz = self.cell_corner_indexes(ix, iy, iz)
        globalidx = self.global_indicies(
            np.dstack([cornersx, cornersy, cornersz]).T)
        return globalidx, inside
    
    def position_to_cell_index(self,pos):
        ix = pos[:, 0] - self.origin[None, 0]
        iy = pos[:, 1] - self.origin[None, 1]
        iz = pos[:, 2] - self.origin[None, 2]
        ix = ix // self.step_vector[None, 0]
        iy = iy // self.step_vector[None, 1]
        iz = iz // self.step_vector[None, 2]
        return ix.astype(int), iy.astype(int), iz.astype(int)
    
    def node_indexes_to_position(self, xindex, yindex, zindex):

        x = self.origin[0] + self.step_vector[0] * xindex
        y = self.origin[1] + self.step_vector[1] * yindex
        z = self.origin[2] + self.step_vector[2] * zindex

        return np.array([x, y, z])
    
    def global_index_to_node_index(self, global_index):
        """
        Convert from global indexes to xi,yi,zi

        Parameters
        ----------
        global_index

        Returns
        -------

        """
        # determine the ijk indices for the global index.
        # remainder when dividing by nx = i
        # remained when dividing modulus of nx by ny is j

        x_index = global_index % self.nsteps[0, None]
        y_index = global_index // self.nsteps[0, None] % \
                  self.nsteps[1, None]
        z_index = global_index // self.nsteps[0, None] // \
                  self.nsteps[1, None]
        return x_index, y_index, z_index
    
    def global_index_to_cell_index(self, global_index):
        """
        Convert from global indexes to xi,yi,zi

        Parameters
        ----------
        global_index

        Returns
        -------

        """
        # determine the ijk indices for the global index.
        # remainder when dividing by nx = i
        # remained when dividing modulus of nx by ny is j

        x_index = global_index % self.nsteps_cells[0, None]
        y_index = global_index // self.nsteps_cells[0, None] % \
                  self.nsteps_cells[1, None]
        z_index = global_index // self.nsteps_cells[0, None] // \
                  self.nsteps_cells[1, None]
        return x_index, y_index, z_index
    def get_neighbour(self,tetra):
        # find out which cell the tetra is in
        cell = tetra // 5
#         cell_x, cell_y, cell_z = self.global_index_to_node_index([cell])
#         print(cell_x,cell_y,cell_z)
#         local_index = tetra % 5
#         if local_index = 
#         neighbours = np.zeros(4)
    def faces(self):
        # there are 5 tetra for every cube 
        ntetra = self.n_cells * 5
        
        faces = np.zeros((ntetra,3))
        
        cell_xi,cell_yi,cell_zi = self.global_index_to_node_index(np.arange(0,self.n_cells)) 
        corners_xi, corners_yi,corners_zi = self.cell_corner_indexes(cell_xi,cell_yi,cell_zi)
        corners = np.dstack([corners_xi,corners_yi,corners_zi])
#         corners = corners.reshape((corners.shape[0]*corners.shape[1],3))
        corners = np.array(corners).swapaxes(0, 2)

        global_index = self.global_indicies(corners)
        tetras = global_index[:,self.tetra_mask]
        
        tetras = tetras.reshape((ntetra,4))
#         for i in range(ntetra):
            
        #print(global_index.shape)
#         print(corners.shape)
#         print(ntetra)
#         print(corners[:,self.tetra_mask,:].shape)
        faces = np.zeros((ntetra*4,3)).astype(int)
        face_tetra = np.zeros((ntetra*4,2)).astype(int)
        fc = build_faces(tetras,faces,face_tetra)
        return faces[0:fc,:], face_tetra[0:fc,:]
    
    def neighbours(self):
        # for each cell
        neighbours = np.zeros((self.ntetra,4)).astype(int)
        neighbours[:] = -1
        for i in range(self.n_cells):
            # get corners
            cell_xi,cell_yi,cell_zi = self.global_index_to_cell_index([i]) 
            corners_xi, corners_yi,corners_zi = self.cell_corner_indexes(cell_xi,cell_yi,cell_zi)
            corners = np.dstack([corners_xi,corners_yi,corners_zi])
            corners = np.array(corners).swapaxes(0, 2)
            global_index = self.global_node_indicies(corners)
            tetras = global_index[:,self.tetra_mask]
#             [0,4,5,3],
#               [4,6,7,5],
#               [0,1,6,4],
#               [0,6,5,4],
#               [0,6,5,2]]
            for j in range(5):
                # first tetra in mask is the centre one so its neighbours are the other ones
                if j == 0:
                    neighbours[i*5+j,:] = i*5+np.arange(1,5)
                    continue
                # add he centre cell to all others
                neighbours[i*5+j,0] = i*5
                if j == 1:
                    pass
                if j == 2:
                    pass
                if j == 3:
                    pass
                if j == 4:
                    pass
        return neighbours

    def tetra(self,i):
        ic = i // 5
        it = i % 5
        cell_xi,cell_yi,cell_zi = self.global_index_to_cell_index([ic]) 
        corners_xi, corners_yi,corners_zi = self.cell_corner_indexes(cell_xi,cell_yi,cell_zi)
        corners = np.dstack([corners_xi,corners_yi,corners_zi])
        tetra  = corners[:,self.tetra_mask[it],:]
        print(tetra)
        print(corners.shape)
        print(ic,it)
#             print(global_index)
#             ntetra = self.n_cells * 5
        
       
#         cell_xi,cell_yi,cell_zi = self.global_index_to_cell_index(np.arange(0,self.n_cells)) 
#         corners_xi, corners_yi,corners_zi = self.cell_corner_indexes(cell_xi,cell_yi,cell_zi)
#         corners = np.dstack([corners_xi,corners_yi,corners_zi])
# #         corners = corners.reshape((corners.shape[0]*corners.shape[1],3))
#         corners = np.array(corners).swapaxes(0, 2)

#         global_index = self.global_node_indicies(corners)
#         print(global_index.shape)
#         print(np.unique(global_index).shape)
#         tetras = global_index[:,self.tetra_mask]
        
#         tetras = tetras.reshape((ntetra,4))
# #         for i in range(ntetra):
#         neighbours = np.zeros((ntetra,4)).astype(int)
#         neighbours[:,:] = -1
#         #print(global_index.shape)
# #         print(corners.shape)
# #         print(ntetra)
# #         print(corners[:,self.tetra_mask,:].shape)
#         #build_neighbours(tetras,neighbours)
#         for i in range(tetra.shape[0]):
#             nn = 0
#             for j in range(tetra.shape[0]):
#                 print(i,j)
#                 if j == i:
#                     continue
#                 n = 0
#                 print(tetra[i,:],tetra[j,:])
#                 for k in range(4):
#                     for l in range(4):
#                         if tetra[i,k] == tetra[j,l]:
#                             n+=1
#                 print(n)
#                 if n==3:
#                     neighbours[i,nn] = j
#             #                 print('n==3',i,nn,j)
#                     nn+=1
#                 if nn == 4:
#             #                 print('nn==3',i,j,n,nn)
#                     break
#                 if n == 2:
#                     continue
#         return neighbours


In [188]:
import time
# nn = []
# elapsed = []
# for n in range(10,30,5):
n = 3
mesh = TetMesh([0,0,0],[n,n,n],[.1,.1,.1])
neigh = mesh.neighbours()
#print(neigh[0])
mesh.tetra(0)
mesh.tetra(neigh[0][1])
#     # print(mesh.nodes)
#     # c = mesh.get_tetra([[.125,.125,.9],
#     #                     [.9, .9, .9],
#     #                     [.9, .9, .125],
#     #                     [.5, .5, .75],
#     #                     [.125, .9, .125],
#     #                     [.125, .9, .125]
#     #                    ])
#     before = time.time()
#     faces, face_tetra = mesh.faces()
#     after = time.time()
#     elapsed.append(after-before)
#     nn.append(mesh.ntetra)

[[[0 0 0]
  [1 1 0]
  [0 1 1]
  [1 0 1]]]
(1, 8, 3)
0 0
[[[1 0 1]
  [1 1 0]
  [1 1 1]
  [0 1 1]]]
(1, 8, 3)
0 2


In [142]:
print(np.sum(np.all(neigh[:,3:] == -1,axis=-1)))

32


In [143]:
print(neigh)

[[ 3 -1 -1 -1]
 [ 3 -1 -1 -1]
 [ 3 -1 -1 -1]
 [ 0  1  2  4]
 [ 3 -1 -1 -1]
 [ 8 -1 -1 -1]
 [ 8 -1 -1 -1]
 [ 8 -1 -1 -1]
 [ 5  6  7  9]
 [ 8 -1 -1 -1]
 [13 -1 -1 -1]
 [13 -1 -1 -1]
 [13 -1 -1 -1]
 [10 11 12 14]
 [13 -1 -1 -1]
 [18 -1 -1 -1]
 [18 -1 -1 -1]
 [18 -1 -1 -1]
 [15 16 17 19]
 [18 -1 -1 -1]
 [23 -1 -1 -1]
 [23 -1 -1 -1]
 [23 -1 -1 -1]
 [20 21 22 24]
 [23 -1 -1 -1]
 [28 -1 -1 -1]
 [28 -1 -1 -1]
 [28 -1 -1 -1]
 [25 26 27 29]
 [28 -1 -1 -1]
 [33 -1 -1 -1]
 [33 -1 -1 -1]
 [33 -1 -1 -1]
 [30 31 32 34]
 [33 -1 -1 -1]
 [38 -1 -1 -1]
 [38 -1 -1 -1]
 [38 -1 -1 -1]
 [35 36 37 39]
 [38 -1 -1 -1]]


In [6]:
15//3

5

In [7]:
c

NameError: name 'c' is not defined

In [ ]:
points = np.random.random((1000000,3))

In [ ]:
#%%snakeviz
c = mesh.get_tetra(points)

In [ ]:
print(c[1][100])

In [ ]:
from LoopStructural.supports import TetMesh

In [ ]:
boundary_points = np.array([[0,0,0],[1,1,1]])
tmesh = TetMesh()
tmesh.setup_mesh(boundary_points,n_tetra = 200000)

In [ ]:
tmesh.elements_for_array(points)

In [92]:
corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                    [0, 0, 1, 0, 0, 1, 1, 1],
                    [0, 0, 0, 1, 1, 1, 0, 1]])
ni = 3
nj = 3
nk = 3
def gi(i,j,k):
    return i+j*(ni+i)+k*(ni+1)*(nj+1)
i = np.arange(0,ni).astype(int)
j = np.arange(0,nj).astype(int)
k = np.arange(0,nk).astype(int)
ii,jj, kk = np.meshgrid(i,j,k,indexing='ij')
iii = ii[:,:,:,None]+corners[None,None,None,0,:]
jjj = jj[:,:,:,None]+corners[None,None,None,1,:]
kkk = kk[:,:,:,None]+corners[None,None,None,2,:]
corner_gi = gi(iii,jjj,kkk)
corner_gi_flat = gi(iii,jjj,kkk)
corner_gi = corner_gi.reshape(ni*nj*nk,8)


tetra = np.vstack([corner_gi[:,[4,6,7,5]],
                    corner_gi[:,[0,1,6,4]],
                    corner_gi[:,[0,6,5,4]],
                    corner_gi[:,[0,6,5,2]],
                    corner_gi[:,[0,4,5,3]],
                    ])

In [112]:
neighbours = np.zeros((tetra.shape[0],5)).astype(int)
neighbours[:] = -1

In [113]:
build_neighbours(tetra,neighbours)

In [116]:
60%5

0

In [114]:
neighbours[:,4] = np.arange(0,tetra.shape[0])
print(neighbours)

[[ 54  -1  -1  -1   0]
 [ 24  55  -1  -1   1]
 [ 25  56  -1  -1   2]
 [ 57 114  -1  -1   3]
 [ 58 115  -1  -1   4]
 [ 59 116  -1  -1   5]
 [ 33  34  87  88   6]
 [ 34  35  88  89   7]
 [ 35  89 125  -1   8]
 [ 63  -1  -1  -1   9]
 [ 64  -1  -1  -1  10]
 [ 65  -1  -1  -1  11]
 [ 66 129  -1  -1  12]
 [ 67 130  -1  -1  13]
 [ 68 131  -1  -1  14]
 [ 69  -1  -1  -1  15]
 [ 70  -1  -1  -1  16]
 [ 71  -1  -1  -1  17]
 [ 72  -1  -1  -1  18]
 [ 73  -1  -1  -1  19]
 [ 74  -1  -1  -1  20]
 [ 75  -1  -1  -1  21]
 [ 76  -1  -1  -1  22]
 [ 77  -1  -1  -1  23]
 [  1  78  -1  -1  24]
 [  2  79  -1  -1  25]
 [ 80  -1  -1  -1  26]
 [ 54  -1  -1  -1  27]
 [ 55 105  -1  -1  28]
 [ 56 106  -1  -1  29]
 [ 57  -1  -1  -1  30]
 [ 58  -1  -1  -1  31]
 [ 59  -1  -1  -1  32]
 [  6 114  -1  -1  33]
 [  6   7 114 115  34]
 [  7   8 115 116  35]
 [ 63  -1  -1  -1  36]
 [ 64  -1  -1  -1  37]
 [ 65  -1  -1  -1  38]
 [ 66  -1  -1  -1  39]
 [ 67  -1  -1  -1  40]
 [ 68  -1  -1  -1  41]
 [ 69  -1  -1  -1  42]
 [ 70  -1  

In [96]:
print(tetra)

[[17  5 21 19]
 [33 21 37 35]
 [49 37 53 51]
 [21  9 25 22]
 [37 25 41 38]
 [53 41 57 54]
 [25 13 29 25]
 [41 29 45 41]
 [57 45 61 57]
 [18  7 23 21]
 [34 23 39 37]
 [50 39 55 53]
 [23 12 28 25]
 [39 28 44 41]
 [55 44 60 57]
 [28 17 33 29]
 [44 33 49 45]
 [60 49 65 61]
 [19  9 25 23]
 [35 25 41 39]
 [51 41 57 55]
 [25 15 31 28]
 [41 31 47 44]
 [57 47 63 60]
 [31 21 37 33]
 [47 37 53 49]
 [63 53 69 65]
 [ 0  1  5 17]
 [16 17 21 33]
 [32 33 37 49]
 [ 3  5  9 21]
 [19 21 25 37]
 [35 37 41 53]
 [ 6  9 13 25]
 [22 25 29 41]
 [38 41 45 57]
 [ 1  2  7 18]
 [17 18 23 34]
 [33 34 39 50]
 [ 5  7 12 23]
 [21 23 28 39]
 [37 39 44 55]
 [ 9 12 17 28]
 [25 28 33 44]
 [41 44 49 60]
 [ 2  3  9 19]
 [18 19 25 35]
 [34 35 41 51]
 [ 7  9 15 25]
 [23 25 31 41]
 [39 41 47 57]
 [12 15 21 31]
 [28 31 37 47]
 [44 47 53 63]
 [ 0  5 19 17]
 [16 21 35 33]
 [32 37 51 49]
 [ 3  9 22 21]
 [19 25 38 37]
 [35 41 54 53]
 [ 6 13 25 25]
 [22 29 41 41]
 [38 45 57 57]
 [ 1  7 21 18]
 [17 23 37 34]
 [33 39 53 50]
 [ 5 12 25

In [115]:
#for i in range(tetra.shape[0]):
i = 60
nn = 0
for j in range(tetra.shape[0]):
    if j == i:
        continue
    n = 0
    print(tetra[i,:],tetra[j,:])
    for k in range(4):
        for l in range(4):
            if tetra[i,k] == tetra[j,l]:
                n+=1
    print(n)
    if n==3:
        neighbours[i,nn] = j
#                 print('n==3',i,nn,j)
        nn+=1
    if nn == 4:
#                 print('nn==3',i,j,n,nn)
        break
    if n == 2:
        continue

[ 6 13 25 25] [17  5 21 19]
0
[ 6 13 25 25] [33 21 37 35]
0
[ 6 13 25 25] [49 37 53 51]
0
[ 6 13 25 25] [21  9 25 22]
2
[ 6 13 25 25] [37 25 41 38]
2
[ 6 13 25 25] [53 41 57 54]
0
[ 6 13 25 25] [25 13 29 25]
5
[ 6 13 25 25] [41 29 45 41]
0
[ 6 13 25 25] [57 45 61 57]
0
[ 6 13 25 25] [18  7 23 21]
0
[ 6 13 25 25] [34 23 39 37]
0
[ 6 13 25 25] [50 39 55 53]
0
[ 6 13 25 25] [23 12 28 25]
2
[ 6 13 25 25] [39 28 44 41]
0
[ 6 13 25 25] [55 44 60 57]
0
[ 6 13 25 25] [28 17 33 29]
0
[ 6 13 25 25] [44 33 49 45]
0
[ 6 13 25 25] [60 49 65 61]
0
[ 6 13 25 25] [19  9 25 23]
2
[ 6 13 25 25] [35 25 41 39]
2
[ 6 13 25 25] [51 41 57 55]
0
[ 6 13 25 25] [25 15 31 28]
2
[ 6 13 25 25] [41 31 47 44]
0
[ 6 13 25 25] [57 47 63 60]
0
[ 6 13 25 25] [31 21 37 33]
0
[ 6 13 25 25] [47 37 53 49]
0
[ 6 13 25 25] [63 53 69 65]
0
[ 6 13 25 25] [ 0  1  5 17]
0
[ 6 13 25 25] [16 17 21 33]
0
[ 6 13 25 25] [32 33 37 49]
0
[ 6 13 25 25] [ 3  5  9 21]
0
[ 6 13 25 25] [19 21 25 37]
2
[ 6 13 25 25] [35 37 41 53]
0
[ 6 13 25 